# BMTC Bus Schedule Converter

---

## Instructions

1. **Upload Your BMTC Bus Schedule Excel File**: Use the file upload widget below to upload your BMTC bus schedule Excel file (.xlsx).
2. **Convert to GTFS**: After uploading the file, click the "Convert to GTFS" button to start the conversion process.
3. **Download Converted GTFS Files**: Once the conversion is complete, a download link will be provided for the zip file containing the converted GTFS files.

---

In [20]:
#############
! pip install -r requirements.txt
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import os
import shutil
import tempfile
import subprocess
import base64
from zipfile import ZipFile

# Define file upload widget
uploader = widgets.FileUpload(accept='.xlsx', multiple=False, description="Upload Excel File")

# Define convert button
convert_button = widgets.Button(description="Convert to GTFS", button_style='primary')

# Define output widget
output = widgets.Output()

# Define download link widget
download_link = widgets.Output()

# Define progress bar
progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', style={'bar_color': 'blue'})

# Function to handle file upload event
def handle_upload(change):
    with output:
        output.clear_output()
        uploaded_filename = next(iter(uploader.value))
        content = uploader.value[uploaded_filename]['content']
        global excel_data
        excel_data = pd.read_excel(content)
        print("File uploaded successfully.")

# Function to convert Excel to GTFS and create a zip file
def convert_to_gtfs(b):
    with output:
        output.clear_output()
        if 'excel_data' not in globals():
            print("Please upload a file first.")
            return
        
        uploaded_filename = next(iter(uploader.value))
        content = uploader.value[uploaded_filename]['content']
        
        try:
            # Write the content to a temporary file
            with tempfile.NamedTemporaryFile(suffix='.xlsx', delete=False) as temp_file:
                temp_file.write(content)
                temp_file_path = temp_file.name
            
            # Define the output folder path
            output_folder = os.path.join(os.getcwd(), "../gtfs_data")
            os.makedirs(output_folder, exist_ok=True)
            
            # Call the conversion script with input file path and output folder path as arguments
            subprocess.run(["python", "../scripts/main.py", temp_file_path, output_folder], check=True)
            
            # Remove the temporary file after conversion
            os.unlink(temp_file_path)
            
            print("Conversion complete.")
            output_zip_file = "../gtfs.zip"

            # Zip the gtfs_data folder
            with ZipFile(output_zip_file, 'w') as zip_file:
                for root, dirs, files in os.walk(output_folder):
                    total_files = len(files)
                    progress_bar.max = total_files
                    for idx, file in enumerate(files, start=1):
                        zip_file.write(os.path.join(root, file), arcname=os.path.relpath(os.path.join(root, file), output_folder))
                        # Update progress bar value
                        progress_bar.value = idx
            
            # Read the contents of the zip file as bytes
            with open(output_zip_file, 'rb') as file:
                zip_data = file.read()

            # Encode the zip data as base64
            b64_zip_data = base64.b64encode(zip_data).decode('utf-8')
            with download_link:
                download_link.clear_output()
                href = f'<a href="data:application/zip;base64,{b64_zip_data}" download={output_zip_file}>Download Converted GTFS</a>'
                display(HTML(href))
                
        except FileNotFoundError as e:
            print("Error: File not found.")
        except subprocess.CalledProcessError as e:
            print("Error: Conversion process failed.")

# Event handlers
uploader.observe(handle_upload, names='value')
convert_button.on_click(convert_to_gtfs)

# Display widgets
display(uploader)
display(convert_button)
display(progress_bar)
display(output)
display(download_link)


FileUpload(value={}, accept='.xlsx', description='Upload Excel File')

Button(button_style='primary', description='Convert to GTFS', style=ButtonStyle())

IntProgress(value=0, description='Progress:', style=ProgressStyle(bar_color='blue'))

Output()

Output()